In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [31]:
#Carregar modelos OPENAI - embedding e chat
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 300)

In [ ]:
# Carregar PDF

pdf_link = os.path.join("..", "data", "os-sertoes.pdf")
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()


In [ ]:
# Separar em chunks
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=600, 
  chunk_overlap=100, 
  length_function=len, 
  add_start_index=True,
  separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)
chunks = text_splitter.split_documents(pages)

In [34]:
# Salvar chunks no ChromaDB
vectordb = Chroma(embedding_function=embeddings, persist_directory='naiveDB')

In [35]:
# Carregar chunks no ChromaDB

naive_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [36]:
compressor = LLMChainExtractor.from_llm(llm)

In [37]:
retriever = ContextualCompressionRetriever(
    base_retriever=naive_retriever,
    base_compressor=compressor
)

In [38]:
TEMPLATE = """
Você é um assistente de IA que ajuda a responder perguntas sobre o conteúdo de documentos legais. Você deve usar o contexto fornecido para responder às perguntas.
Query: {question}
Context: {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [41]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": RunnableLambda(lambda x: retriever.invoke(x["question"]))})

output_parser = StrOutputParser()

compressor_retrievel_chain = (
    setup_retrieval
    | rag_prompt
    | llm
    | output_parser
)

In [42]:
resposta = compressor_retrievel_chain.invoke({
    "question": "Quais são os principais desafios éticos associados à IA?"
})
print(resposta)

Os principais desafios éticos associados à IA incluem questões de privacidade e proteção de dados, viés e discriminação algorítmica, transparência e explicabilidade dos algoritmos, responsabilidade e accountability, e o impacto da IA no mercado de trabalho e na sociedade como um todo. É importante considerar essas questões éticas ao desenvolver e implementar tecnologias de IA.
